# Exercise 4: Wind-generated set-up and circulations in a homogeneous lake

Wind over a lake can generate a water level set-up and drive both horizontal and vertical circulations. In this exercise we study set-up and circulations in a lake with uniform density. We investigate the effects of depth variation, and consider consequences of modelling this situation in either 2DH or 3D.

The lake covers an area of 10 km by 10 km and is 3m deep. The westerly wind blows steadily with $U_{10}$ = 15.0 m/s.


## 4.4) Wind over lake with non-uniform depth (3D)

a)	We also investigate the lake with dredged channel using a 3D model. Set the parameters to simulate the same situation as in 2DH and run the model. One figure shows a vertical cross section of the horizontal velocity. Where is the near bed current the greatest?


In [1]:
# Imports
import bmi
import bmi.wrapper
import ipywidgets as widgets
from ipywidgets import HBox, VBox, interactive, Layout, interact
import numpy as np
import time
import os
# Something very fishy happening if I don't define a plot first.
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable

%matplotlib notebook
import matplotlib.pyplot as plt
import sys
plt.ioff()
_ = plt.figure()
plt.ion()

In [2]:
# Toggle button for hiding the raw code
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>

<style>
.widget-box {
  overflow: visible;
}
</style>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [3]:
%run config-Ex4D.py

In [4]:
data = []
items = []
wrapper = bmi.wrapper.BMIWrapper(engine=engine, configfile=mdufile);
wrapper.initialize();
# wrapper.update()

In [5]:
maxsteps = 100
parameters = [
    {
        "parameter": "wx",
        "description": "Wind speed $[m/s]$",
        "default": 15
    },    {
        "parameter": "frcu",
        "description": r"Bottom Roughness (Chézy) $[\sqrt{m}/s]$",
        "default": str(wrapper.get_var("frcu")[0])
    }
]

In [6]:
# for i in range(wrapper.get_var_count()):
#     print(wrapper.get_var_name(i), wrapper.get_var_shape(wrapper.get_var_name(i)))

In [7]:
# Create widgets
style = {'description_width': 'initial'}

run = widgets.Button(
    description='Run model',
    button_style='',
    icon='play'
)
update = widgets.Button(
    description='Single update',
    button_style='',
    tooltip='Update with 1 timestep',
    icon='step-forward'
)
restart = widgets.Button(
    description='Restart model',
    button_style='',
    tooltip='Restart entire model with initial inputs',
    icon='retweet'
)

settings = widgets.HTML(
    value="Welcome!",
    placeholder='Input settings'
)

play = widgets.Play(
#     interval=10,
    value=0,
    min=0,
    max=int(wrapper.get_end_time()),
    step=1,
    description="Press play",
    disabled=False
)


nsteps = widgets.BoundedIntText(
    description="Number of timesteps",
    value=100,
    min=0,
    max=maxsteps,
    style=style,
    layout=Layout(width='15vw')
)

slider = widgets.IntSlider(  
    min=0,
    max=1,
    value=0,
    continuous_update=False
)

widgets.jslink((play, 'value'), (slider, 'value'))
player = widgets.HBox([play, slider])

# parameters = ["zbndz", "zbndq", "frcu"]
# p_descrip = ["Boundary Water level", "Boundary Discharge", "Roughness Chézy"]

items=[]
for p in parameters: 
    items.append(widgets.Text(
        description=p["description"],
        disabled=False,
        value=str(p["default"]),
        placeholder=p["parameter"],
        style=style,
        layout=Layout(width='50vw')
    ))

In [8]:
  
# Model specific function 
xz = wrapper.get_var('xz')[:]
yz = wrapper.get_var('yz')[:]
indy = np.argsort(yz)[::-1]
newx = xz[indy]

randind = np.random.choice(len(xz), int(len(xz)/5), replace=False)

lX = len(np.unique(xz))
lY = len(np.unique(yz))
newx = newx.reshape((lY, lX))
indx = np.argsort(newx, axis=1)

Z = int(wrapper.get_var_shape('ucx')[0] / (lX * lY))
# print(Z)

xunq = np.unique(xz)
x_slice = np.tile(xunq, (Z, 1)).transpose()
XYshape = lX * lY
def grid2matrix(arr):
    new_arr = arr.copy()
    new_arr = new_arr[indy]
    new_arr = new_arr.reshape((lY, lX))

    c = np.zeros_like((new_arr))

    for i, x in enumerate(new_arr):
        c[i, :] = new_arr[i, indx[i]]
    return c

#     Automate parameter and grid sizes
def update_data():
    ucx = wrapper.get_var('ucx')[:]
    ucy = wrapper.get_var('ucy')[:]
    zws = wrapper.get_var('zws')[:]
    s1 = wrapper.get_var("s1")[:]
    
    new_mag = np.zeros((lX, lY, Z))
    new_magy = np.zeros((lX, lY, Z))
    new_zws = np.zeros((lX, lY, Z))
    for layer in range(Z-1):
        new_mag[:, :, layer] = grid2matrix(ucx[XYshape:][layer::11])
        new_magy[:, :, layer] = grid2matrix(ucy[XYshape:][layer::11])
        new_zws[:, :, layer] = grid2matrix(zws[XYshape:][layer::11])
    
    UX = grid2matrix(ucx[XYshape:][0::11])
    UY = grid2matrix(ucy[XYshape:][0::11])
    
    data.append(dict({
        "time": wrapper.get_current_time(),
        "ucx": UX, 
        "ucy": UY, 
        "top_ucx": new_mag[:, :, 9],
        "top_ucy": new_magy[:, :, 9],
        "slice_ucx": new_mag[:, 50, :], 
        "zws": new_zws[:, 50, :], 
        "top_s1": grid2matrix(s1.copy())
    }))
    slider.max = len(data) -1
    settings.value = "Model update, timestep: {}".format(data[-1]["time"])
update_data()


In [9]:
# plt.figure()
# plt.plot(xz, yz, '.')

In [10]:
# zws = data[-1]["zws"]
# xunq = np.unique(xz)
# X = np.tile(xunq, (12, 1))
# colors = data[-1]["slice_ucx"]
# plt.figure()
# a= plt.pcolormesh(X.transpose(), zws, colors);
# plt.colorbar(a, cmap="seismic")

In [11]:
# print(list(wrapper.get_var('zws')[:]
#     ))

In [12]:
# # mag0 = wrapper.get_var('ucmag')[(lX*lY):][0:-1:11]
# # mag1 = wrapper.get_var('ucmag')[(lX*lY):][1:-2:11]

# # n = grid2matrix(mag1)
# plt.figure()
# plt.imshow(data[-1]["slice_ucx"].transpose())
# plt.colorbar()

In [13]:
# np.zeros((lX, lY, Z))

In [14]:
# plt.figure()
# ucx = wrapper.get_var('ucx')[:]
# ucy = wrapper.get_var('ucy')[:]
# mag = np.sqrt(ucx.copy()**2 + ucy.copy()**2)
# # vortscat = axs.scatter(xz, yz, marker="s", c=data[0]["magR"], vmin=0, vmax=2)

# sc = plt.scatter(xz, yz, c=mag)
# # plt.colorbar(sc)

In [15]:
# Standard functions for button widgets
dostop = False
def update_model(b=None):
#     Update the model with t = 1
    wrapper.update(wrapper.get_time_step())
    if(data[-1]["time"] != wrapper.get_current_time()):
        update_data()

def start_loop(n):
#     Start the loop for running the model continuously
#     while run.value == True: 
    for i in range(n):
        update_model()
        if (wrapper.get_current_time() >= wrapper.get_end_time()):
            stop_model()
            break
        if dostop == True: 
            stop_model()
            break
    stop_model()

def run_model(change=None): 
#     When the run/stop model button is pressed either start the model loop or stop it
    run.disabled = True
    update.disabled = True
    restart.disabled = True
    settings.disabled = True
    nsteps.disabled = True
    for i in items:
        i.disabled = True
    start_loop(int(nsteps.value))
#         thread = threading.Thread(target=start_loop)
#         thread.start()

def stop_model(change=None):
    update.disabled = False
    restart.disabled = False
    run.disabled = False
    nsteps.disabled = False
    for i in items:
        i.disabled = False
    dostop = True

def change_param(v):
    try:
        v = v.owner
    except AttributeError: 
        v = v
    if v.value == "":
        return
    try:
        float(v.value)
        old_par = wrapper.get_var(v.placeholder)
        new_par = np.ones_like(old_par) * float(v.value)
        wrapper.set_var(v.placeholder, new_par)
        settings.value = "Value ({}) has been set to: {}".format(v.description, new_par[0])
    except ValueError:
        v.value = str(next((x['default'] for x in parameters if x['parameter'] == v.placeholder), None))
        settings.value = "Not a correct input for {}".format(v.description)

def start_model():
#     start model
    wrapper = bmi.wrapper.BMIWrapper(engine=engine, configfile=mdufile)
    wrapper.initialize()
    update_data()
    for i in range(len(items)): 
        items[i].value = str(parameters[i]['default'])
        
def restart_model(b=None):
#     stop the model and call function to restart the model
    del data[:]
    slider.value = 0
    wrapper.finalize()
    start_model()
    settings.value = "Restarting model"

In [16]:
# plt.imshow()

In [ ]:
step = 1
# set plot size
plt.rcParams["figure.figsize"] = (9, 7) # (w, h)

# link functions to widgets
run.on_click(run_model)
update.on_click(update_model)
restart.on_click(restart_model)

def set_plot(change):
    t = change['new']
    minU = data[-1]["top_ucx"].min()
    maxU = data[-1]["top_ucx"].max()
    minS = data[-1]["top_s1"].min()
    maxS = data[-1]["top_s1"].max()
    
#     print(maxV, minV)
    ux_top.set_clim(vmin= -round(1.2 *  max(abs(minU), abs(maxU)), 2), vmax=round(1.2 *  max(abs(minU), abs(maxU)), 2))
    ux_side.set_clim(vmin= -round(1.2 *  max(abs(minU), abs(maxU)), 2), vmax=round(1.2 *  max(abs(minU), abs(maxU)), 2))
    s1_top.set_clim(vmin= -round(1.2 *  max(abs(minS), abs(maxS)), 2), vmax=round(1.2 *  max(abs(minS), abs(maxS)), 2))
    vel_line.set_data(data[t]['slice_ucx'][int(lX / 2), :], data[t]['zws'][int(lX / 2), :])
    ax3.set_xlim(-round(2.5 *  max(abs(minU), abs(maxU)), 2), round(2.5 *  max(abs(minU), abs(maxU)), 2))
    if (t < len(data)):
        ux_top.set_data(data[t]["top_ucx"])
        s1_top.set_data(data[t]["top_s1"])
        ux_side.set_array(data[t]["slice_ucx"][:-1, :-1].ravel())
        t1.set_text("time: " + str(data[t]["time"]) + "[s]")
        ucx_flattened = data[t]["top_ucx"].ravel()
        ucy_flattened = data[t]["top_ucy"].ravel()
        quiver.set_UVC(ucx_flattened[randind],ucy_flattened[randind])
        fig.canvas.draw()
        
slider.observe(set_plot, 'value')

controls = HBox([run, update, restart])
params = VBox(items)

for i in items: 
    change_param(i)
    i.observe(change_param, names='value')

display(VBox([settings, HBox([nsteps, controls]), HBox([params])]))
fig = plt.figure()
ax0 = plt.subplot2grid((2, 2), (0, 0), colspan=1)
ax1 = plt.subplot2grid((2, 2), (0, 1), colspan=1)
ax2 = plt.subplot2grid((2, 2), (1, 0), rowspan=1)
ax3 = plt.subplot2grid((2, 2), (1, 1), rowspan=1)
# ax4 = plt.subplot2grid((2, 3), (1, 2), rowspan=1)
display(player)

t1 = fig.suptitle("time: " + str(data[0]["time"]) + "[s]")

ux_top = ax0.imshow(data[0]["top_ucx"], extent=[xz.min(), xz.max(), yz.min(), yz.max()], cmap="seismic")
ax0.set_xlabel("X $[m]$")
ax0.set_ylabel("Y $[m]$")

ucx_flattened = data[0]["top_ucx"].ravel()
ucy_flattened = data[0]["top_ucy"].ravel()
quiver = ax0.quiver(xz[randind], yz[randind], ucx_flattened[randind], ucy_flattened[randind], angles='xy', scale_units='xy', scale=0.0005)
divider = make_axes_locatable(ax0)
cax0 = divider.append_axes('right', size='5%', pad=0.05)
cbar = plt.colorbar(ux_top, cax=cax0);
ax0.set_title('Surface velocity ($x$-direction) $[m/s]$')
sliceline, = ax0.plot((5000, 5000), (0, 10000), "--", color="#ff33cc")

s1_top = ax1.imshow(data[0]["top_s1"], extent=[xz.min(), xz.max(), yz.min(), yz.max()])
ax1.set_title('Water level $[m]$')
divider1 = make_axes_locatable(ax1)
cax1 = divider1.append_axes('right', size='5%', pad=0.05)
cbar1 = plt.colorbar(s1_top, cax=cax1);
ax1.set_ylabel("Y $[m]$")
ax1.set_xlabel("X $[m]$")

ux_side = ax2.pcolormesh(x_slice, data[0]["zws"], data[-1]["slice_ucx"], vmin=-0.01, vmax=0.01, cmap="seismic")
divider2 = make_axes_locatable(ax2)
cax2 = divider2.append_axes('right', size='5%', pad=0.05)
cbar2 = plt.colorbar(ux_side, cax=cax2);
sliceline2, = ax2.plot((5000, 5000), (-8, 0), "--", color="#ff33cc")
ax2.set_ylabel("Z $[m]$")
ax2.set_xlabel("Y $[m]$")

ax3.yaxis.set_visible(True)
vel_line, = ax3.plot(data[0]['slice_ucx'][int(lX / 2), :], data[0]['zws'][int(lX / 2), :])
# ax4.yaxis.set_visible(False)
ax3.set_ylabel("Z $[m]$")
ax3.set_xlabel("Velocity in $x$ direction $[m/s]$")
ax3.grid()
plt.draw()


<IPython.core.display.Javascript object>

In [ ]:
wrapper.get_var('wx')
# plt.figure()
# # ux_side = 
# # print(np.shape(x_slice), np.shape(data[0]["zws"]), np.shape(data[0]["slice_ucx"]))
# ux_side = plt.pcolormesh(x_slice, data[0]["zws"], data[0]["slice_ucx"], cmap="seismic")
# ux_side.set_facecolors(data[-1]["slice_ucx"])
# plt.colorbar(ux_side)
# print(np.shape(data[0]['slice_ucx']))
# print(np.shape(data[0]['zws']))

In [ ]:
# plt.figure()
# plt.plot(data[0]['ucx'][int(lX / 2), :], data[0]['zws'][int(lX / 2), :])

In [ ]:
# plt.figure()
# ax1 = plt.subplot2grid((2, 3), (0, 0), colspan=3)
# ax2 = plt.subplot2grid((2, 3), (1, 0), rowspan=1)
# ax3 = plt.subplot2grid((2, 3), (1, 1), rowspan=1, sharey=ax2)
# ax3.yaxis.set_visible(False)
# ax4 = plt.subplot2grid((2, 3), (1, 2), rowspan=1, sharey=ax2)
# ax4.yaxis.set_visible(False)


In [ ]:
# plt.figure()
# plt.imshow(data[0]["top_s1"], extent=[xz.min(), xz.max(), yz.min(), yz.max()])
# plt.colorbar()

In [ ]:
# wrapper.get_var('wx')